In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

In [ ]:
# 1. Load Libraries and Utilities
from langchain_groq import ChatGroq
from teammerge.utils import load_esco_data, make_skill_lookup

In [36]:
# 2. Load ESCO Data
from teammerge.utils import ESCOContext

context = ESCOContext.load("data/esco")

### Embed and Save Skills to ChromaDB

In [ ]:
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize embedding model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2", show_progress=True)

# Convert skills_df to documents
skill_documents = [
    Document(
        page_content=row['description'],
        metadata={
            "uri": row['conceptUri'],
            "label": row['preferredLabel']
        }
    )
    for _, row in context.skills_df.iterrows()
]

# Create Chroma index
skill_vectorstore = Chroma.from_documents(
    documents=skill_documents,
    embedding=embedding_model,
    persist_directory="chroma/skills_index"
)

skill_vectorstore.persist()

Batches:   0%|          | 0/171 [00:00<?, ?it/s]

Batches:   0%|          | 0/171 [00:00<?, ?it/s]

Batches:   0%|          | 0/95 [00:00<?, ?it/s]

/tmp/ipykernel_208497/2514182280.py:27: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  skill_vectorstore.persist()


### Embed and Save Occupations to ChromaDB

In [ ]:
occupation_documents = [
    Document(
        page_content=row['description'],
        metadata={
            "uri": row['conceptUri'],
            "label": row['preferredLabel']
        }
    )
    for _, row in context.occupations_df.iterrows()
]

occupation_vectorstore = Chroma.from_documents(
    documents=occupation_documents,
    embedding=embedding_model,
    persist_directory="chroma/occupations_index"
)

occupation_vectorstore.persist()

Batches:   0%|          | 0/95 [00:00<?, ?it/s]

In [19]:
# 4. Load ChromaDB Vectorstores
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

skill_vectorstore = Chroma(persist_directory="./chroma/skills_index", embedding_function=embedding_model)
occupation_vectorstore = Chroma(persist_directory="./chroma/occupations_index", embedding_function=embedding_model)

/tmp/ipykernel_208497/230648347.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  skill_vectorstore = Chroma(persist_directory="./chroma/skills_index", embedding_function=embedding_model)


In [20]:
# 6. Initialize LLM
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")

In [54]:
# 7. Create Agents
from teammerge.tools import get_skill_tool, get_occupation_tool, get_skill_gap_tool
from teammerge.agents import build_hr_agent, build_esco_agent, build_planner_agent, build_hr_chain, build_planner_chain

esco_tools = [
    get_skill_tool(skill_vectorstore),
    get_occupation_tool(occupation_vectorstore),
    get_skill_gap_tool(context)
]

hr_agent = build_hr_chain(llm)
planner_agent = build_planner_chain(llm)
esco_agent = build_esco_agent(llm, tools=esco_tools)  # Skill search only


In [55]:
# 8. Example Usage: roundtable
# The roundtable discussion simulates a meeting where agents discuss team composition and skill gaps.

from teammerge.agents import run_roundtable

initial_prompt="""
We currently have 2 data engineers and 3 business analysts on staff. 
The incoming team brings 2 data scientists and 1 machine learning engineers. 
Let's explore role overlaps and potential gaps.
"""


In [56]:
results = run_roundtable(
    initial_prompt=initial_prompt,
    hr_agent=hr_agent,
    esco_agent=esco_agent,
    planner_agent=planner_agent
)

🟢 Initial Prompt:
 
We currently have 2 data engineers and 3 business analysts on staff. 
The incoming team brings 2 data scientists and 1 machine learning engineers. 
Let's explore role overlaps and potential gaps.



> Entering new LLMChain chain...
Prompt after formatting:
You are an HR expert. Based on the following team update, describe the current and incoming team structure in 2–3 concise sentences.

Team update:

We currently have 2 data engineers and 3 business analysts on staff. 
The incoming team brings 2 data scientists and 1 machine learning engineers. 
Let's explore role overlaps and potential gaps.



> Finished chain.

🧑‍💼 HR Agent:
 Here is a concise summary of the current and incoming team structure:

The current team consists of 2 Data Engineers and 3 Business Analysts. The incoming team will add 2 Data Scientists and 1 Machine Learning Engineer, expanding the team's technical capabilities. The combined team will have a mix of technical and analytical roles, with pot

ValueError: One input key expected got ['esco', 'hr']